In [ ]:
import cv2
import numpy as np
from pynput.mouse import Button ,Controller
import wx

In [ ]:
import speech_recognition as sr
import pyttsx3 as p
from ipynb.fs.full.info_wiki import info
from ipynb.fs.full.movie_reviews import *
from ipynb.fs.full.music import *
from ipynb.fs.full.recomendations import *
from ipynb.fs.full.translations import *

In [ ]:
mouse=Controller()

app=wx.App(False) #initialize obj of wx
(sx,sy)=wx.GetDisplaySize() #size of monitor

(camx,camy)=(320,240) #camera will capture img in this resolution
cap=cv2.VideoCapture(0) #attaching our in-built camera 0
cap.set(3,camx) #setting width of the frame
cap.set(4,camy) #setting height of the frame

#range for HSV-color/saturation(1 grey)/value(brightness) (green color)
lower_g=np.array([160,0,0])
upper_g=np.array([180,255,255])

#Kernel - filters for convolution
kernelOpen=np.ones((5,5))
kernelClose=np.ones((20,20))

mLocOld=np.array([0,0]) #old mouse location
mouseLoc=np.array([0,0]) #new mouse location after applying damping

DampingFactor=5 #Damping factor must be greater than 1

isPressed=0
openx,openy,openw,openh=(0,0,0,0)

In [ ]:
#     #initial conversation
#     rl = sr.Recognizer()
#     engine = p.init()
#     engine.say("hello sir, how are you doing today?")
#     engine.runAndWait()
#     with sr.Microphone() as source:
#         rl.adjust_for_ambient_noise(source)
#         print("check1")
#         audio = rl.listen(source)
#         try:
#             text = rl.recognize_google(audio)
#             print(text)
#         except sr.UnknownValueError:
#             print("")
#         except sr.RequestError as e:
#             print("")
    
#     #cue for giving instructions
#     engine.say("what would you like me to do?")
#     engine.runAndWait()
#     print("check2")
    
#     #giving instructions
#     r2 = sr.Recognizer()
#     with sr.Microphone() as source:
#         r2.adjust_for_ambient_noise(source)
#         audio = r2.listen(source)
#         try:
#             instruction = r2.recognize_google(audio)
#             print(text)
#         except sr.UnknownValueError:
#             print("")
#         except sr.RequestError as e:
#             print("")
            
#     #getting info from wiki
#     r3 = sr.Recognizer()
#     if "information" in instruction:
#         engine.say("information about what?")
#         engine.runAndWait()
#         with sr.Microphone() as source1:
#             audio2 = r3.listen(source1)
#             try:
#                 information = r3.recognize_google(audio2)
#                 bot = info()
#                 bot.get_info(information)
#             except sr.UnknownValueError:
#                 print("")
#             except sr.RequestError as e:
#                 print("")       
    
#     #movie rating
#     r4 = sr.Recognizer()
#     if "review" in instruction:
#         engine.say("what is the name of the movie?")
#         engine.runAndWait()
#         with sr.Microphone() as source2:
#             audio3 = r4.listen(source2)
#             try:
#                 rating = r4.recognize_google(audio3)
#                 bot = Movie()
#                 bot.movie_review(rating)
#             except sr.UnknownValueError:
#                 print("")
#             except sr.RequestError as e:
#                 print("")
                
#     #playing music
#     r5 = sr.Recognizer()
#     if "music" in instruction:
#         engine.say("which artist you want me to play?")
#         engine.runAndWait()
#         with sr.Microphone() as source3:
#             audio4 = r5.listen(source3)
#             try:
#                 video = r5.recognize_google(audio4)
#                 bot = music()
#                 bot.play(video)
#             except sr.UnknownValueError:
#                 print("")
#             except sr.RequestError as e:
#                 print("")   
                
#     #movie recomendation
#     if "recommendation" in instruction:
#         engine.say("here are the list of movies you can choose to watch from?")
#         engine.runAndWait()
#         bot = recom()
#         bot.recom_info() 
        
#    #transalation
#     r6 = sr.Recognizer()
#     if "meaning" in instruction:
#         engine.say("Which word sir?")
#         engine.runAndWait()
#         with sr.Microphone() as source4:
#             audio5 = r6.listen(source4)
#             try:
#                 word1 = r6.recognize_google(audio5)
#                 translate(word1)
#             except sr.UnknownValueError:
#                 print("")
#             except sr.RequestError as e:
#                 print("") 

In [ ]:

while True:

    ret,img=cap.read() #capturing live images

    imgHSV=cv2.cvtColor(img,cv2.COLOR_BGR2HSV) #converting to another color space
    mask=cv2.inRange(imgHSV,lower_g,upper_g) #converting it to a range of green color

    #using morphology to erase noise as maximum as possible 
    new_mask=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernelOpen)
    another_mask=cv2.morphologyEx(new_mask,cv2.MORPH_CLOSE,kernelClose)
    final_mask=another_mask
    
    conts,h=cv2.findContours(final_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)#boundary lines for constant green color objects

    # Once 2 objects are detected the center of there distance will be the reference on controlling the mouse
    if(len(conts)==2):

        #if the button is pressed we need to release it first (continous clicking issue)
        if(isPressed==1):
            isPressed=0
            mouse.release(Button.left)

        #drawing the rectagle around both objects
        x1,y1,w1,h1=cv2.boundingRect(conts[0])
        x2,y2,w2,h2=cv2.boundingRect(conts[1])
        cv2.rectangle(img,(x1,y1),(x1+w1,y1+h1),(255,0,0),2)
        cv2.rectangle(img,(x2,y2),(x2+w2,y2+h2),(255,0,0),2)

        #the line between the center of the previous rectangles
        cx1=int(x1+w1/2)
        cy1=int(y1+h1/2)
        cx2=int(x2+w2/2)
        cy2=int(y2+h2/2)
        cv2.line(img,(cx1,cy1),(cx2,cy2),(255,0,0),2)

        #the center of that line (reference point)
        clx=int((cx1+cx2)/2)
        cly=int((cy1+cy2)/2)
        cv2.circle(img,(clx,cly),2,(0,0,255),2)

        #adding the damping factor so that the movement of the mouse is smoother
        mouseLoc=mLocOld+((clx,cly)-mLocOld)/DampingFactor
        mouse.position=(sx-int((mouseLoc[0]*sx)/camx),int((mouseLoc[1]*sy)/camy))#passing damped mouse location
        while mouse.position!=(sx-int((mouseLoc[0]*sx)/camx),int((mouseLoc[1]*sy)/camy)):
            pass #wait till captured image resolution is not converted to our window(screen) size resolution

        #setting the old location to the current mouse location
        mLocOld=mouseLoc

        #these variables were added so that we get the outer rectangle that combines both objects 
        openx,openy,openw,openh=cv2.boundingRect(np.array([[[x1,y1],[x1+w1,y1+h1],[x2,y2],[x2+w2,y2+h2]]]))

    #when there's only one object detected it will act as a left click mouse and will keep on clicking which we don't want   
    elif(len(conts)==1):
        x,y,w,h=cv2.boundingRect(conts[0])

        # we check first and we allow the press fct if it's not pressed yet
        #we did that to avoid the continues pressing 
        if(isPressed==0):

            if(abs((w*h-openw*openh)*100/(w*h))<30): #the difference between the combined rectangle for both objct and the 
                isPressed=1                          #the outer rectangle is not more than 30%
                mouse.press(Button.left)
                openx,openy,openw,openh=(0,0,0,0)

        #this else was added so that if there's only one object detected it will not act as a mouse  
        else:
            #getting rectangle coordinates and drawing it 
            x,y,w,h=cv2.boundingRect(conts[0])
            cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)

            #getting the center of the circle that will be inside the outer rectangle
            cx=int(x+w/2)
            cy=int(y+h/2)
            cv2.circle(img,(cx,cy),int((w+h)/4),(0,0,255),2)#drawing that circle

            mouseLoc=mLocOld+((cx,cy)-mLocOld)/DampingFactor
            mouse.position=(sx-int((mouseLoc[0]*sx)/camx),int((mouseLoc[1]*sy)/camy))#passing damped mouse location
            while mouse.position!=(sx-int((mouseLoc[0]*sx)/camx),int((mouseLoc[1]*sy)/camy)):
                pass #wait till captured image resolution is not converted to our window size resolution
            mLocOld=mouseLoc

        

    #showing the results 
    cv2.imshow("Virtual mouse",img)
    
    #waiting for 'W' to be pressed to quit 
    if cv2.waitKey(1) & 0xFF==ord('w'):
        break

cap.release()
cv2.destroyAllWindows()